In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Esophageal_Cancer"
cohort = "GSE107754"

# Input paths
in_trait_dir = "../../input/GEO/Esophageal_Cancer"
in_cohort_dir = "../../input/GEO/Esophageal_Cancer/GSE107754"

# Output paths
out_data_file = "../../output/preprocess/Esophageal_Cancer/GSE107754.csv"
out_gene_data_file = "../../output/preprocess/Esophageal_Cancer/gene_data/GSE107754.csv"
out_clinical_data_file = "../../output/preprocess/Esophageal_Cancer/clinical_data/GSE107754.csv"
json_path = "../../output/preprocess/Esophageal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A novel genomic signature predicting FDG uptake in diverse metastatic tumors"
!Series_summary	"Purpose: Building a universal genomic signature predicting the intensity of FDG uptake in diverse metastatic tumors may allow us to understand better the biological processes underlying this phenomenon and their requirements of glucose uptake."
!Series_summary	"Methods: A balanced training set (n=71) of metastatic tumors including some of the most frequent histologies, with matched PET/CT quantification measurements and whole human genome gene expression microarrays, was used to build the signature. Selection of microarray features was carried out exclusively on the basis of their strong association with FDG uptake (as measured by SUVmean35) by means of univariate linear regression. A thorough bioinformatics study of these genes was performed and multivariable models were built by fitting several state of the art regression techniques to the training set

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import re
import json
from typing import Optional, Callable, Dict, Any

# 1. Analyze gene expression data availability
# Based on the background information, this appears to be gene expression microarray data
# The series summary mentions "whole human genome gene expression microarrays"
is_gene_available = True

# 2. Analyze clinical features
# 2.1 Data Availability
# For trait - Esophageal Cancer, look at row 2 where tissue types are mentioned
trait_row = 2  # Row containing tissue information

# For gender, it's in row 0
gender_row = 0  # Row containing gender information

# For age, I don't see age information in the sample characteristics
age_row = None  # Age data not available

# 2.2 Data Type Conversion functions
def convert_trait(value):
    """Convert tissue type to binary for Esophageal Cancer."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract the value after colon
    match = re.search(r':\s*(.+)', value)
    if not match:
        return None
    
    tissue_value = match.group(1).strip().lower()
    
    # Check if it's esophageal cancer (binary classification)
    if 'esophagus cancer' in tissue_value:
        return 1
    else:
        return 0

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract the value after colon
    match = re.search(r':\s*(.+)', value)
    if not match:
        return None
    
    gender_value = match.group(1).strip().lower()
    
    if 'female' in gender_value:
        return 0
    elif 'male' in gender_value:
        return 1
    else:
        return None

def convert_age(value):
    """Placeholder function for age conversion (not used as age is not available)."""
    return None

# 3. Save metadata for initial filtering
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None

# Initial validation
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # Load or create clinical data in the correct format
    # The sample characteristics dictionary is not in the right format for direct conversion to DataFrame
    # We need to find an existing clinical data file or properly structure the data
    
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path)
    else:
        # Create a proper DataFrame structure where columns are samples and rows are features
        # Assuming we need to create it from the sample_dict
        sample_dict = {
            0: ['gender: Male', 'gender: Female'],
            1: ['dataset: Validation set', 'dataset: Training set'],
            2: ['biopsy location: Lung', 'biopsy location: Lymph node', 'biopsy location: Primary', 
                'biopsy location: Liver', 'biopsy location: Retroperitoneal implant', 
                'tissue: Pancreatic cancer', 'tissue: Esophagus cancer', 'tissue: Breast cancer', 
                'tissue: Colorectal cancer', 'tissue: Ovarian cancer', 'tissue: Head&neck cancer', 
                'tissue: Lung cancer', 'tissue: Malignant Melanoma', 'tissue: Endometrial cancer', 
                'tissue: Cervix cancer', 'tissue: Soft tissue sarcoma', 'tissue: Gastric cancer', 
                'tissue: Unknown primary', 'tissue: Malignant Mesothelioma', 'tissue: Thyroid cancer', 
                'tissue: Testes cancer', 'tissue: Non Hodgkin lymphoma', 'tissue: Merkel cell carcinoma', 
                'tissue: Vaginal cancer', 'tissue: Kidney cancer', 'tissue: Cervical cancer', 
                'tissue: Bile duct cancer', 'tissue: Urothelial cancer'],
            3: ['suvmean35: 4.09', 'suvmean35: 8.36', 'suvmean35: 5.18', 'suvmean35: 10.74', 
                'suvmean35: 8.62', 'suvmean35: 8.02', 'suvmean35: 6.87', 'suvmean35: 4.93', 
                'suvmean35: 1.96', 'suvmean35: 8.83', 'suvmean35: 3.96', 'suvmean35: 3.38', 
                'suvmean35: 9.95', 'suvmean35: 5.19', 'suvmean35: 7.22', 'suvmean35: 5.02', 
                'suvmean35: 4.92', 'suvmean35: 4.99', 'suvmean35: 4.01', 'suvmean35: 2.52', 
                'suvmean35: 5.52', 'suvmean35: 8.38', 'suvmean35: 3.46', 'suvmean35: 4.07', 
                'suvmean35: 4.67', 'suvmean35: 7.09', 'suvmean35: 4.83', 'suvmean35: 6.7', 
                'suvmean35: 3.95', 'suvmean35: 5.03']
        }
        
        # Create an index of feature rows
        index = list(sample_dict.keys())
        
        # Find the maximum number of samples (columns) needed
        max_cols = max(len(values) for values in sample_dict.values())
        
        # Create an empty DataFrame with the right dimensions
        clinical_data = pd.DataFrame(index=index, columns=range(max_cols))
        
        # Fill in the DataFrame
        for row, values in sample_dict.items():
            for col, value in enumerate(values):
                clinical_data.loc[row, col] = value
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        gender_row=gender_row,
        convert_gender=convert_gender,
        age_row=age_row,
        convert_age=convert_age
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features)
    print(f"Preview of clinical features: {preview}")
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Preview of clinical features: {0: [0.0, 1.0], 1: [0.0, 0.0], 2: [0.0, nan], 3: [0.0, nan], 4: [0.0, nan], 5: [0.0, nan], 6: [1.0, nan], 7: [0.0, nan], 8: [0.0, nan], 9: [0.0, nan], 10: [0.0, nan], 11: [0.0, nan], 12: [0.0, nan], 13: [0.0, nan], 14: [0.0, nan], 15: [0.0, nan], 16: [0.0, nan], 17: [0.0, nan], 18: [0.0, nan], 19: [0.0, nan], 20: [0.0, nan], 21: [0.0, nan], 22: [0.0, nan], 23: [0.0, nan], 24: [0.0, nan], 25: [0.0, nan], 26: [0.0, nan], 27: [0.0, nan], 28: [nan, nan], 29: [nan, nan]}
Clinical data saved to: ../../output/preprocess/Esophageal_Cancer/clinical_data/GSE107754.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 74
Header line: "ID_REF"	"GSM2878070"	"GSM2878071"	"GSM2878072"	"GSM2878073"	"GSM2878074"	"GSM2878075"	"GSM2878076"	"GSM2878077"	"GSM2878078"	"GSM2878079"	"GSM2878080"	"GSM2878081"	"GSM2878082"	"GSM2891194"	"GSM2891195"	"GSM2891196"	"GSM2891197"	"GSM2891198"	"GSM2891199"	"GSM2891200"	"GSM2891201"	"GSM2891202"	"GSM2891203"	"GSM2891204"	"GSM2891205"	"GSM2891206"	"GSM2891207"	"GSM2891208"	"GSM2891209"	"GSM2891210"	"GSM2891211"	"GSM2891212"	"GSM2891213"	"GSM2891214"	"GSM2891215"	"GSM2891216"	"GSM2891217"	"GSM2891218"	"GSM2891219"	"GSM2891220"	"GSM2891221"	"GSM2891222"	"GSM2891223"	"GSM2891224"	"GSM2891225"	"GSM2891226"	"GSM2891227"	"GSM2891228"	"GSM2891229"	"GSM2891230"	"GSM2891231"	"GSM2891232"	"GSM2891233"	"GSM2891234"	"GSM2891235"	"GSM2891236"	"GSM2891237"	"GSM2891238"	"GSM2891239"	"GSM2891240"	"GSM2891241"	"GSM2891242"	"GSM2891243"	"GSM2891244"	"GSM2891245"	"GSM2891246"	"GSM2891247"	"GSM2891248"	"GSM2891249"	"GSM2891250"	"GSM2891251"	"GSM2891252"	"GSM2891253"	

Index(['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056',
       'A_23_P100074', 'A_23_P100092', 'A_23_P100103', 'A_23_P100111',
       'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156',
       'A_23_P100177', 'A_23_P100189', 'A_23_P100196', 'A_23_P100203',
       'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100263'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the output
# The identifiers (like A_23_P100001) appear to be Agilent microarray probe IDs
# These are not standard human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE107754
Line 6: !Series_title = A novel genomic signature predicting FDG uptake in diverse metastatic tumors
Line 7: !Series_geo_accession = GSE107754
Line 8: !Series_status = Public on Jan 22 2018
Line 9: !Series_submission_date = Dec 06 2017
Line 10: !Series_last_update_date = Jan 23 2019
Line 11: !Series_pubmed_id = 29349517
Line 12: !Series_summary = Purpose: Building a universal genomic signature predicting the intensity of FDG uptake in diverse metastatic tumors may allow us to understand better the biological processes underlying this phenomenon and their requirements of glucose uptake.
Line 13: !Series_summary = Methods: A balanced training set (n=71) of metastatic tumors including some of the mos


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan

### Step 6: Gene Identifier Mapping

In [7]:
# Let's first rerun the gene annotation extraction with the proper function
# We'll try using the library function now that we understand the file structure
gene_annotation = get_gene_annotation(soft_file)

# From the preview, we can see:
# - 'ID' column in the annotation contains probe IDs that match expression data index
# - 'GENE_SYMBOL' column contains the human gene symbols we need to map to

# 1. Extract the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'GENE_SYMBOL')

# 2. Print a preview of the mapping to verify
print("Gene mapping preview:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level expression to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# 4. Print the dimensions and preview of the gene expression data
print(f"Gene expression data shape: {gene_data.shape}")
print("Gene expression data preview (first 5 genes, first 5 samples):")
print(gene_data.iloc[:5, :5])

# 5. Save the gene expression data to the specified output file
out_gene_dir = os.path.dirname(out_gene_data_file)
os.makedirs(out_gene_dir, exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


Gene mapping preview:
             ID     Gene
0  A_23_P100001  FAM174B
1  A_23_P100011    AP3S2
2  A_23_P100022     SV2B
3  A_23_P100056   RBPMS2
4  A_23_P100074     AVEN
Gene expression data shape: (18488, 84)
Gene expression data preview (first 5 genes, first 5 samples):
          GSM2878070  GSM2878071  GSM2878072  GSM2878073  GSM2878074
Gene                                                                
A1BG       16.675251   18.589095   17.665959   20.260758   17.885257
A1BG-AS1    8.138944    9.361230    8.513994    9.269932    9.537095
A1CF       15.194191   15.252929   16.779446   19.487898   15.190126
A2LD1       9.231020    9.510204    9.701203    8.614287    8.349443
A2M        14.936963   14.432973   15.230273   14.494223   15.789791


Gene expression data saved to: ../../output/preprocess/Esophageal_Cancer/gene_data/GSE107754.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (18247, 84)
First few genes with their expression values after normalization:
          GSM2878070  GSM2878071  GSM2878072  GSM2878073  GSM2878074  \
Gene                                                                   
A1BG       16.675251   18.589095   17.665959   20.260758   17.885257   
A1BG-AS1    8.138944    9.361230    8.513994    9.269932    9.537095   
A1CF       15.194191   15.252929   16.779446   19.487898   15.190126   
A2M        14.936963   14.432973   15.230273   14.494223   15.789791   
A2ML1      10.363561   11.081253    9.592718   10.258384   11.055192   

          GSM2878075  GSM2878076  GSM2878077  GSM2878078  GSM2878079  ...  \
Gene                                                                  ...   
A1BG       23.469022   17.237170   21.554459   18.427697   19.441237  ...   
A1BG-AS1    9.026286    8.261324    9.419955    8.954172    8.822408  ...   
A1CF       19.816419   15.124037   18.691166   15.129180   15.509904  ...   
A2M 

Normalized gene data saved to ../../output/preprocess/Esophageal_Cancer/gene_data/GSE107754.csv
Raw clinical data shape: (4, 85)
Clinical features:


                   GSM2878070  GSM2878071  GSM2878072  GSM2878073  GSM2878074  \
Esophageal_Cancer         0.0         0.0         0.0         0.0         0.0   
Gender                    1.0         0.0         1.0         1.0         0.0   

                   GSM2878075  GSM2878076  GSM2878077  GSM2878078  GSM2878079  \
Esophageal_Cancer         0.0         0.0         0.0         0.0         0.0   
Gender                    1.0         0.0         0.0         0.0         0.0   

                   ...  GSM2891255  GSM2891256  GSM2891257  GSM2891258  \
Esophageal_Cancer  ...         0.0         0.0         0.0         0.0   
Gender             ...         1.0         1.0         1.0         1.0   

                   GSM2891259  GSM2891260  GSM2891261  GSM2891262  GSM2891263  \
Esophageal_Cancer         0.0         0.0         0.0         0.0         0.0   
Gender                    0.0         1.0         0.0         1.0         1.0   

                   GSM2891264  
Esophageal_Ca

Data shape after handling missing values: (84, 18249)
For the feature 'Esophageal_Cancer', the least common label is '1.0' with 4 occurrences. This represents 4.76% of the dataset.
The distribution of the feature 'Esophageal_Cancer' in this dataset is severely biased.

For the feature 'Gender', the least common label is '1.0' with 35 occurrences. This represents 41.67% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data was determined to be unusable or empty and was not saved
